In [1]:
import sys
sys.path.append("../../xcorr")


In [33]:
DB.close()

import xcorr_db
import importlib
importlib.reload(xcorr_db)


<module 'xcorr_db' from '../../xcorr/xcorr_db.py'>

In [34]:
DB = xcorr_db.setup_db('./initial.db')

In [35]:
hpo_id = DB.insert_hpo_record(2)

TypeError: sequence item 0: expected str instance, int found

In [131]:
import pandas as pd
import csv
import numpy as np
import json

# Generate Stats From hpos.txt #

Each hpo writes an hpo.txt with hpo_id, params, run_dir, ts, and val loss

In [209]:
df = pd.read_csv('~/Documents/results/cp1/non_nci_hpo_log/hpos.txt', sep="|", header=None, names=["i", "hpo_id", "params", "run_dir", "ts", "val_loss"])

In [212]:
df.groupby("hpo_id")['val_loss'].agg([np.min, np.max, np.mean, np.std])


,amin,amax,mean,std
hpo_id,,,,
1,0.325794,2.205444e+34,3.968941e+31,9.029660e+32
2,0.278948,1.036991e+31,1.955343e+28,3.718788e+29
3,0.321078,5.450333e+32,7.536106e+29,1.865691e+31
4,0.269695,3.885900e+33,4.450476e+30,1.295403e+32
5,0.326156,1.407330e+32,1.564757e+29,4.691097e+30
6,0.325993,5.586196e+33,6.231631e+30,1.862063e+32
7,0.332956,2.311167e+30,5.317878e+27,8.994357e+28
8,0.343334,5.819235e+31,9.109562e+28,1.979917e+30
9,0.333361,1.440266e+30,4.994766e+27,6.875268e+28


In [161]:
n = 10
smallest = df.groupby('hpo_id')['val_loss'].nsmallest(n)
best_n = df.iloc[smallest.index.get_level_values(1), :]
#best_n.to_csv('~/Documents/results/cp1/best_{}_nci.txt'.format(n), sep="|", index=False)
params = '/home/nick/Documents/results/cp1/best_{}_nci_params.txt'.format(n)
# Write out the best n parameters
best_n[['params']].to_csv(params, index=False, 
                        header=False, quoting=csv.QUOTE_NONE, sep='|')

In [102]:
pd.__version__


'0.22.0'

In [141]:
# stats for the best n
best_n.groupby("hpo_id")['val_loss'].agg([np.min, np.max, np.mean, np.std])

,amin,amax,mean,std
hpo_id,,,,
1,0.201724,0.206538,0.203615,0.001687
2,0.179843,0.188933,0.185792,0.003026
3,0.186955,0.196299,0.191846,0.003553
4,0.184138,0.194240,0.190929,0.003104
5,0.184527,0.196454,0.192813,0.003435


## Generate upf -- one json param dict per line -- file from the best n runs ##

In [165]:
from os import path

with open(params) as f_in:
    lines = f_in.readlines()
    
upf = '/home/nick/Documents/results/cp1/best_{}_nci_params_upf.txt'.format(n)
with open(upf, 'w') as f_out:
    for line in lines:
        j = json.loads(line)
        train_sources = j['train_sources']
        if 'cell_feature_subset_path' in j and train_sources == 'NCI60':
            fsp = path.basename(j['cell_feature_subset_path'])
            train = '{}_train.h5'.format(fsp[:fsp.find('_features')])
            j['use_exported_data'] = '/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/cache/{}'.format(train)
            
            
        del j['save_path']
        j['epochs'] = 100
        f_out.write('{}\n'.format(json.dumps(j)))

## Create upf for creating test, train generated data ##

In [189]:


js = """{{"batch_size": 6144, "train_sources": "{}", "preprocess_rnaseq": "combat", "gpus": "0 1 2 3 4 5", "cell_feature_subset_path": "/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/xcorr_data/{}_{}_2000_1000_features.txt", "export_data": "/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/cache/{}_{}_2000_1000_{}.h5", "no_feature_source": true, "no_response_source": true, "cp": true}}"""



studies = ['CCLE', 'CTRP', 'gCSI', 'GDSC']
for s1 in studies:
    for s2 in studies:
        if s1 != s2:
            j1 = js.format(s1, s1, s2, s1, s2, 'train')
            j2 = js.format(s2, s1, s2, s1, s2, 'test')
            #print(j1)
            #print(j2)
            
for s1 in studies:
    s2 = 'NCI60'
    j = js.format(s2, s1, s2, s1, s2, 'test')
    print(j)
            
            

{"batch_size": 6144, "train_sources": "NCI60", "preprocess_rnaseq": "combat", "gpus": "0 1 2 3 4 5", "cell_feature_subset_path": "/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/xcorr_data/CCLE_NCI60_2000_1000_features.txt", "export_data": "/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/cache/CCLE_NCI60_2000_1000_test.h5", "no_feature_source": true, "no_response_source": true, "cp": true}
{"batch_size": 6144, "train_sources": "NCI60", "preprocess_rnaseq": "combat", "gpus": "0 1 2 3 4 5", "cell_feature_subset_path": "/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/xcorr_data/CTRP_NCI60_2000_1000_features.txt", "export_data": "/autofs/nccs-svm1_proj/med106/ncollier/repos/Supervisor/workflows/cp1/cache/CTRP_NCI60_2000_1000_test.h5", "no_feature_source": true, "no_response_source": true, "cp": true}
{"batch_size": 6144, "train_sources": "NCI60", "preprocess_rnaseq": "combat", "gpus": "0 1 2 3 4 5", "cell_feature_subset_path

## Create upf for inferencing runs, using inputs.txt as produced by training upf ##

In [208]:
from os import path
# python uno_infer.py --data CTRP_CCLE_2000_1000_test.h5 --model_file model.h5 
inputs = '/home/nick/Documents/results/cp1/inputs.txt'


model_class_ids = {}
next_id = 0

infer_upf = '/home/nick/Documents/repos/Supervisor/workflows/cp1/data/infer_upf_all.txt'
with open(infer_upf, 'w') as f_out:
    studies = ['CCLE', 'CTRP', 'gCSI', 'GDSC', 'NCI60']
    with open(inputs) as f_in:
        reader = csv.reader(f_in, delimiter="|")
        for r in reader:
            params = json.loads(r[2])
            save_path = params['save_path']
            if 'cell_feature_subset_path' in params:
                fsp = path.basename(params['cell_feature_subset_path'])
                fsp_prefix = fsp[:fsp.find('_features')]
                test_data = '{}_test.h5'.format(fsp_prefix)
                save_path = params['save_path']
                train_source = params['train_sources']
                #if fsp.find('_NCI60_') != -1:
                f_out.write('{},{},{}\n'.format(test_data, save_path, fsp_prefix))
            else:
                train_source = params['train_sources']
                for s in studies:
                    test_data = '{}.h5'.format(s)
                    f_out.write('{},{},{}_{}\n'.format(test_data, save_path, train_source, s))
        
        
        


In [185]:
f = "/home/nick/Documents/repos/Supervisor/workflows/cp1/scripts/counts_by_hpo.csv"
hp = {}

class Entry:

    def __init__(self, start):
        self.start = start
        self.end = -1
    
    def __repr__(self):
        return "[{}, {}]".format(self.start, self.end)

with open(f) as f_in:
    reader = csv.reader(f_in)
    next(reader)
    for row in reader:
        if row[1] == "1":
            hpo_id = int(row[3])
            h = float(row[2])
            if not hpo_id in hp:
                hp[hpo_id] = [Entry(h)]
            else:
                entry = hp[hpo_id][-1]
                if entry.end != -1:
                    hp[hpo_id].append(Entry(h))
        elif row[1] == "0":
            hpo_id = int(row[3])
            h = float(row[2])
            hp[hpo_id][-1].end = h


with open('/home/nick/Documents/repos/Supervisor/workflows/cp1/scripts/start_end.csv', 'w') as f_out:
    for k in hp:
        for e in hp[k]:
            f_out.write('{},{},{}\n'.format(e.start, e.end, k))



In [6]:
import csv
import os

f = "/home/nick/Documents/results/cp1/inference_log.txt"
prefix = '/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/'
with open(f) as f_in:
    reader = csv.reader(f_in, delimiter='|')
    for i, row in enumerate(reader):
        #train_path = row[2]
        run_id = int(os.path.basename(os.path.dirname(row[2])))
        if run_id < 200:
            print('{}{}'.format(prefix, i))
        

/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/0
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/1
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/2
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/3
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/4
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/5
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/6
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/7
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/8
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/9
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/10
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/11
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/12
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all_4/run/13
/gpfs/alpine/med106/scratch/ncollier/experiments/infer_all